In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
from wandb.integration.langchain import WandbTracer

In [5]:
WandbTracer.init({"project": "langchain_constituciones", "tags":["hyde"]})

wandb: Streaming LangChain activity to W&B at https://wandb.ai/capecape/langchain_constituciones/runs/94qixipr
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [6]:
base_embeddings = OpenAIEmbeddings()
llm = OpenAI()

we may need to play with the `prompt_template` here as it's pretty basic

In [7]:
prompt_template = """
Porfavor responde las pregunta con respecto a las constituciones de Chile y España. 
Si la informacion esta disponible, da una respuesta para cada Chile y otra para España.
Pregunta: {question}
Respuesta:"""
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

Let's use some more fancy embeddings like [HyDE](https://python.langchain.com/en/latest/modules/chains/index_examples/hyde.html)

In [8]:
embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=base_embeddings)

## Using HyDE

In [9]:
files = ["./data/Constitucion_cl.pdf", "./data/Constitucion_es.pdf"]

Let's load documents one by one just for fun

In [10]:
# for some reason it returns a list with 1 element!
docs = [UnstructuredPDFLoader(f).load()[0] for f in files]

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
splitted_docs = text_splitter.split_documents(docs)

we need to recreate the db as the old one didn't use HyDE

In [12]:
db = FAISS.from_documents(splitted_docs, embeddings)

`k=5` means it retrieves only the top 5 documents

In [13]:
retriever = db.as_retriever(search_kwargs=dict(k=5))

let's test

In [14]:
query = "Cuanto dura el mandato de un diputado"

In [15]:
db.similarity_search(query, k=5)

[Document(page_content='los demás en proporción a la población.\n\n3.    La elección se verificará en cada circunscripción atendiendo a\n\ncriterios de representación proporcional.\n\n4.    El Congreso es elegido por cuatro años. El mandato de los\n\nDiputados termina cuatro años después de su elección o el día de la\n\ndisolución de la Cámara.\n\n5.    Son electores y elegibles todos los españoles que estén en\n\npleno uso de sus derechos políticos.\n\nLa ley reconocerá y el Estado facilitará el ejercicio del derecho de\n\nsufragio a los españoles que se encuentren fuera del territorio de\n\nEspaña.\n\nEl mandato\n\nparlamentario\n\nEl Congreso de\n\nlos Diputados\n\nSistema\n\nelectoral\n\nCuatro años de\n\nlegislatura\n\n23\n\n6.    Las elecciones tendrán lugar entre los treinta días y sesenta\n\ndías desde la terminación del mandato. El Congreso electo deberá\n\nser convocado dentro de los veinticinco días siguientes a la celebra-\n\nción de las elecciones.\n\nArtículo 69.\n\n1.   

## Question answering

Let's put some ChatGPT...

In [16]:
model_name = 'gpt-3.5-turbo'

qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(model_name=model_name), 
                                           retriever, 
                                           return_source_documents=True)

let's keep track of the previous messages in the history

In [17]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [18]:
len(result["source_documents"])

5

In [19]:
result["answer"]

'El mandato de un diputado en España dura cuatro años después de su elección o el día de la disolución de la Cámara. Esto se establece en el Artículo 68.6 de la Constitución española.'

In [20]:
query = "Como es la division territorial en Chile?"

this is slow...

In [21]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [22]:
len(result["source_documents"])

5

In [23]:
result["answer"]

'En Chile, el territorio se divide en regiones y estas a su vez en provincias, y las provincias en comunas. La creación, supresión y denominación de regiones, provincias y comunas; la modificación de sus límites, así como la fijación de las capitales de las regiones y provincias, serán materia de ley orgánica constitucional. Además, el gobierno regional estará constituido por el intendente y el consejo regional, y para el gobierno y administración interior del Estado se divide en regiones, provincias y comunas. La administración local de cada comuna o agrupación de comunas que determine la ley reside en una municipalidad, la que estará constituida por el alcalde y el concejo.'

In [24]:
WandbTracer.finish()

wandb: Finished uploading data to W&B at https://wandb.ai/capecape/langchain_constituciones/runs/94qixipr
